Environment: `py39_torch271`

In [ ]:
import os
import random
from copy import deepcopy

from typing import Optional
from dataclasses import dataclass, asdict
from pathlib import Path
from datetime import datetime

import torch
import numpy as np
import pandas as pd
from dacite import from_dict
from sklearn.model_selection import train_test_split

from pyrutils.torch.train_utils import train, save_checkpoint
from pyrutils.torch.multi_task import MultiTaskLossLearner
from vhoi.data_loading import (
    input_size_from_data_loader, 
    select_model_data_feeder, 
    select_model_data_fetcher,
)
from vhoi.data_loading_custom import (
    create_data,
    create_data_loader
)
from vhoi.losses_custom_v2 import (
    select_loss, 
    decide_num_main_losses, 
    select_loss_types, 
    select_loss_learning_mask,
)
from vhoi.models import load_model_weights
from vhoi.models_custom_v2 import TGGCN

seed = 42
random.seed(seed)   # Python的随机性
os.environ['PYTHONHASHSEED'] = str(seed)    # 设置Python哈希种子，为了禁止hash随机化，使得实验可复现
np.random.seed(seed)   # numpy的随机性
torch.manual_seed(seed)   # torch的CPU随机性，为CPU设置随机种子
torch.cuda.manual_seed(seed)   # torch的GPU随机性，为当前GPU设置随机种子
torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU.   torch的GPU随机性，为所有GPU设置随机种子
torch.backends.cudnn.benchmark = False   # if benchmark=True, deterministic will be False
torch.backends.cudnn.deterministic = True   # 选择确定性算法

In [ ]:
class DictMixin:
    def get(self, key, default_value=None):
        return getattr(self, key, default_value)

    def as_dict(self):
        return asdict(self)
    
@dataclass
class Resources(DictMixin):
    use_gpu: bool
    num_threads: int

@dataclass
class ModelMetadata(DictMixin):
    model_name: str
    input_type: str

@dataclass
class ModelParameters(DictMixin):
    add_segment_length: int
    add_time_position: int
    time_position_strategy: str
    positional_encoding_style: str
    attention_style: str
    bias: bool
    cat_level_states: int
    discrete_networks_num_layers: int
    discrete_optimization_strategy: str
    filter_discrete_updates: bool
    gcn_node: int
    hidden_size: int
    message_humans_to_human: bool
    message_human_to_objects: bool
    message_objects_to_human: bool
    message_objects_to_object: bool
    message_geometry_to_objects: bool
    message_geometry_to_human: bool
    message_segment: bool
    message_type: str
    message_granularity: str
    message_aggregation: str
    object_segment_update_strategy: str
    share_level_mlps: int
    update_segment_threshold: float

@dataclass
class ModelOptimization(DictMixin):
    batch_size: int
    clip_gradient_at: float
    epochs: int
    learning_rate: float
    val_fraction: float

@dataclass
class BudgetLoss(DictMixin):
    add: bool
    human_weight: float
    object_weight: float

@dataclass
class SegmentationLoss(DictMixin):
    add: bool
    pretrain: bool
    sigma: float
    weight: float

@dataclass
class ModelMisc(DictMixin):
    anticipation_loss_weight: float
    budget_loss: BudgetLoss
    first_level_loss_weight: float
    impose_segmentation_pattern: int
    input_human_segmentation: bool
    input_object_segmentation: bool
    make_attention_distance_based: bool
    multi_task_loss_learner: bool
    pretrained: bool
    pretrained_path: Optional[str]
    segmentation_loss: SegmentationLoss

@dataclass
class ModelLogging(DictMixin):
    root_log_dir: str
    checkpoint_name: str
    log_dir: str

@dataclass
class Models(DictMixin):
    metadata: ModelMetadata
    parameters: ModelParameters
    optimization: ModelOptimization
    misc: ModelMisc
    logging: ModelLogging

@dataclass
class Data(DictMixin):
    name: str
    path: str
    path_zarr: str
    path_obb_zarr: str
    path_hbb_zarr: str
    path_hps_zarr: str
    cross_validation_test_subject: str
    scaling_strategy: Optional[str]
    downsampling: int

@dataclass
class Config(DictMixin):
    resources: Resources
    models: Models
    data: Data
    
metadata_dict = {
    "model_name": "2G-GCN",
    "input_type": "multiple"
}

parameters_dict = {
    "add_segment_length": 0,  # length of the segment to the segment-level rnn. 0 is off and 1 is on.
    "add_time_position": 0,  # absolute time position to the segment-level rnn. 0 is off and 1 is on.
    "time_position_strategy": "s",  # input time position to segment [s] or discrete update [u].
    "positional_encoding_style": "e",  # e [embedding] or p [periodic].
    "attention_style": "v3",  # v1 [concat], v2 [dot-product], v3 [scaled_dot-product], v4 [general]
    "bias": True,
    "cat_level_states": 0,  # concatenate first and second level hidden states for predictors MLPs.
    "discrete_networks_num_layers": 1,  # depth of the state change detector MLP.
    "discrete_optimization_strategy": "gs",  # straight-through [st] or gumbel-sigmoid [gs]
    "filter_discrete_updates": False,  # maxima filter for soft output of state change detector.
    "gcn_node": 25,  # custom, original: 19 for cad120, 30 for bimanual, 26 for mphoi
    "hidden_size": 512,  # 512 for cad120 & mphoi; 64 for bimanual
    "message_humans_to_human": False, # custom, original: True
    "message_human_to_objects": True,
    "message_objects_to_human": True,
    "message_objects_to_object": False, # custom, original: True
    "message_geometry_to_objects": True,
    "message_geometry_to_human": True,  # custom, original: False
    "message_segment": True,
    "message_type": "v2",  # v1 [relational] or v2 [non-relational]
    "message_granularity": "v1",  # v1 [generic] or v2 [specific]
    "message_aggregation": "att",  # mean_pooling [mp] or attention [att]
    "object_segment_update_strategy": "ind",  # same_as_human [sah], independent [ind], or conditional_on_human [coh]
    "share_level_mlps": 0,  # whether to share [1] or not [0] the prediction MLPs of the levels.
    "update_segment_threshold": 0.5  # [0.0, 1.0)
}

optimization_dict = {
    "batch_size": 32,  # mphoi:8; cad120:16; bimanual: 32
    "clip_gradient_at": 0.0,
    "epochs": 10, # custom, original: cad120 & mphoi:40; bimanual: 60
    "learning_rate": 1e-4,  # mphoi:1e-4; cad120 & bimanual:1e-3
    "val_fraction": 0.2,
}

data_dict = {
    "name": "mphoi",
    "path": f"{os.getcwd()}/data/MPHOI/MPHOI/mphoi_ground_truth_labels.json",
    "path_zarr": f"{os.getcwd()}/data/MPHOI/MPHOI/mphoi_derived_features/faster_rcnn.zarr",
    "path_obb_zarr": f"{os.getcwd()}/data/MPHOI/MPHOI/mphoi_derived_features/object_bounding_boxes.zarr",
    "path_hbb_zarr": f"{os.getcwd()}/data/MPHOI/MPHOI/mphoi_derived_features/human_bounding_boxes.zarr",
    "path_hps_zarr": f"{os.getcwd()}/data/MPHOI/MPHOI/mphoi_derived_features/human_pose.zarr",
    "cross_validation_test_subject": "Subject14",  # Subject45, Subject25, Subject14
    "scaling_strategy": None,  # null or "standard"
    "downsampling": 1 # custom, original: 3, 1 = full FPS, 2 = half FPS, ...
}

# root_log_dir = f"{os.getcwd()}/outputs_hiergat/{data_dict['name']}/{metadata_dict['model_name']}"
root_log_dir = f"{os.getcwd()}/outputs_hiergat/custom"
checkpoint_name = (
    f"hs{parameters_dict['hidden_size']}_e{optimization_dict['epochs']}_bs{optimization_dict['batch_size']}_"
    f"lr{optimization_dict['learning_rate']}_{parameters_dict['update_segment_threshold']}_{datetime.now().strftime('%Y%m%d%H%M%S')}"
)
log_dir = f"{root_log_dir}/{checkpoint_name}"
print("Log directory:", log_dir)
os.makedirs(log_dir, exist_ok=True)

cfg_dict = {
    "resources": {
        "use_gpu": True,
        "num_threads": 32
    },
    "models": {
        "metadata": metadata_dict,
        "parameters": parameters_dict,
        "optimization": optimization_dict,
        "misc": {
            "anticipation_loss_weight": 1.0,
            "budget_loss": {
                "add": False,
                "human_weight": 1.0,
                "object_weight": 1.0
            },
            "first_level_loss_weight": 0.0,  # if positive, first level does frame-level prediction
            "impose_segmentation_pattern": 1,  # 0 [no pattern], 1 [all ones]
            "input_human_segmentation": False,  # (was "flase" in YAML, corrected here)
            "input_object_segmentation": False,
            "make_attention_distance_based": True,  # only meaningful if message_aggregation is attention
            "multi_task_loss_learner": False,
            "pretrained": False,  # unfortunately need two entries for checkpoint name
            "pretrained_path": None,  # specified parameters must match pre-trained model
            "segmentation_loss": {
                "add": False,
                "pretrain": False,
                "sigma": 0.0,  # Gaussian smoothing
                "weight": 1.0
            }
        },
        "logging": {
            "root_log_dir": root_log_dir,
            "checkpoint_name": checkpoint_name,
            "log_dir": log_dir
        },
    },
    "data": data_dict,
}

cfg = from_dict(data_class=Config, data=cfg_dict)

torch.set_num_threads(cfg.resources.num_threads)
model_name, model_input_type = cfg.models.metadata.model_name, cfg.models.metadata.input_type
batch_size, val_fraction = cfg.models.optimization.batch_size, cfg.models.optimization.val_fraction
misc_dict = cfg.get('misc', default_value={})
sigma = misc_dict.get('segmentation_loss', {}).get('sigma', 0.0)
scaling_strategy = cfg.data.scaling_strategy
downsampling = cfg.data.downsampling

action_classes = [
    # human
    'supervise',        # 0
    'collaborate with', # 1
    'assist',           # 2
    'lead',             # 3
    'coordinate with',  # 4
    'listen to',        # 5

    # rebar
    'tie',              # 6
    'erect',            # 7
    'prepare_rebar',    # 8
    'transport',        # 9

    # formwork
    'install',          # 10
    'prepare_formwork', # 11

    # concrete
    'pour',             # 12
    'finish',           # 13

    # equipment         
    'use',              # 14
    'carry',            # 15

    # all
    'inspect',          # 16
    'no interaction',   # 17
]
new_action_classes = [
    # rebar
    'tie',              # 0
    'erect',            # 1
    'prepare_rebar',    # 2
    'transport',        # 3

    # equipment         
    'use',              # 4
    'carry',            # 5

    # all
    'inspect',          # 6
    'no interaction',   # 7
]
# num_classes = len(action_classes)
num_classes = len(new_action_classes)

# ================================
# Features for seq_len = 3 seconds
# ================================
train_features_dirs = [
    '/root/vs-gats-plaster/deepsort/hiergat_data_v3_3s/C0071_full_MP4_anno_for_labelling_done_faridz_full_temporal_3s/features', # {0: 130, 2: 90, 6: 105, 7: 223}
    '/root/vs-gats-plaster/deepsort/hiergat_data_v3_3s/C0074_full_MP4_anno_for_labelling_done_ray_full_temporal_3s/features', # {0: 124, 1: 25, 2: 11, 6: 55, 7: 133}
    '/root/vs-gats-plaster/deepsort/hiergat_data_v3_3s/C0078_full_MP4_anno_for_labelling_done_anne_full_temporal_3s/features', # {0: 130, 7: 65} 
    '/root/vs-gats-plaster/deepsort/hiergat_data_v3_3s/C0083_full_MP4_anno_for_labelling_done_putu_full_temporal_3s/features', # {0: 240, 1: 5, 2: 56, 3: 1, 4: 6, 6: 2, 7: 34}
    '/root/vs-gats-plaster/deepsort/hiergat_data_v3_3s/C0085_full_MP4_anno_for_labelling_done_yoga_full_temporal_3s/features', # {0: 200, 2: 181, 4: 27, 7: 84}
    '/root/vs-gats-plaster/deepsort/hiergat_data_v3_3s/C0087_full_MP4_anno_for_labelling_done_arga_full_temporal_3s/features', # {0: 205, 5: 8, 6: 134, 7: 236}
    '/root/vs-gats-plaster/deepsort/hiergat_data_v3_3s/C0090_full_MP4_anno_for_labelling_rizky_full_temporal_3s/features', # {}
    '/root/vs-gats-plaster/deepsort/hiergat_data_v3_3s/C0098_full_MP4_anno_for_labelling_done_akbar_full_temporal_3s/features', # {0: 2, 1: 10, 5: 23, 7: 84}
    '/root/vs-gats-plaster/deepsort/hiergat_data_v3_3s/C0101_full_MP4_anno_for_labelling_done_faridz_full_temporal_3s/features', # {0: 147, 6: 5, 7: 309}
    '/root/vs-gats-plaster/deepsort/hiergat_data_v3_3s/C0105_full_MP4_anno_for_labelling_full_temporal_3s/features', # {}
    '/root/vs-gats-plaster/deepsort/hiergat_data_v3_3s/C0106_full_MP4_anno_for_labelling_full_temporal_3s/features', # {}
    '/root/vs-gats-plaster/deepsort/hiergat_data_v3_3s/C0108_full_MP4_anno_for_labelling_fixed_full_temporal_3s/features', # {1: 26, 3: 21, 4: 35, 6: 16, 7: 157}
    '/root/vs-gats-plaster/deepsort/hiergat_data_v3_3s/C0110_full_MP4_anno_for_labelling_full_temporal_3s/features', # {2: 23, 3: 29, 4: 25, 7: 264}
    
    # Previously used as validation
    '/root/vs-gats-plaster/deepsort/hiergat_data_v3_3s/C0100_full_MP4_anno_for_labelling_done_putu_full_temporal_3s/features', # {0: 23, 2: 10, 4: 2, 7: 191}
    '/root/vs-gats-plaster/deepsort/hiergat_data_v3_3s/C0109_full_MP4_anno_for_labelling_full_temporal_3s/features', # {1: 1541, 3: 38, 4: 35}       
    
    # Previously used as testing
    '/root/vs-gats-plaster/deepsort/hiergat_data_v3_3s/C0104_full_MP4_anno_for_labelling_full_temporal_3s/features', # {}
    '/root/vs-gats-plaster/deepsort/hiergat_data_v3_3s/C0099_full_MP4_anno_for_labelling_done_arga_full_temporal_3s/features', # {0: 4, 2: 274, 5: 5, 6: 6, 7: 25}
]

# ================================
# Features for seq_len = 2 seconds
# ================================
# train_features_dirs = [
#     '/root/vs-gats-plaster/deepsort/hiergat_data_v2512_2s/C0071_full_MP4_anno_for_labelling_done_faridz_full_temporal_2s/features', # {0: 276, 2: 193, 6: 250}
#     '/root/vs-gats-plaster/deepsort/hiergat_data_v2512_2s/C0074_full_MP4_anno_for_labelling_done_ray_full_temporal_2s/features',    # {0: 256, 1: 40, 2: 20, 4: 1, 6: 114}
#     '/root/vs-gats-plaster/deepsort/hiergat_data_v2512_2s/C0078_full_MP4_anno_for_labelling_done_anne_full_temporal_2s/features',   # {0: 254}
#     '/root/vs-gats-plaster/deepsort/hiergat_data_v2512_2s/C0083_full_MP4_anno_for_labelling_done_putu_full_temporal_2s/features',   # {0: 451, 1: 15, 2: 118, 3: 2, 4: 12, 6: 4}
#     '/root/vs-gats-plaster/deepsort/hiergat_data_v2512_2s/C0085_full_MP4_anno_for_labelling_done_yoga_full_temporal_2s/features',   # {0: 365, 2: 291, 4: 44}
#     '/root/vs-gats-plaster/deepsort/hiergat_data_v2512_2s/C0087_full_MP4_anno_for_labelling_done_arga_full_temporal_2s/features',   # {0: 429, 5: 13, 6: 292}
#     '/root/vs-gats-plaster/deepsort/hiergat_data_v2512_2s/C0090_full_MP4_anno_for_labelling_rizky_full_temporal_2s/features',       # {0: 3}
#     '/root/vs-gats-plaster/deepsort/hiergat_data_v2512_2s/C0098_full_MP4_anno_for_labelling_done_akbar_full_temporal_2s/features',  # {0: 7, 1: 31, 4: 2, 5: 61}
#     '/root/vs-gats-plaster/deepsort/hiergat_data_v2512_2s/C0101_full_MP4_anno_for_labelling_done_faridz_full_temporal_2s/features', # {0: 258, 2: 9, 6: 13}
#     '/root/vs-gats-plaster/deepsort/hiergat_data_v2512_2s/C0105_full_MP4_anno_for_labelling_full_temporal_2s/features',             # {}
#     '/root/vs-gats-plaster/deepsort/hiergat_data_v2512_2s/C0106_full_MP4_anno_for_labelling_full_temporal_2s/features',             # {}
#     '/root/vs-gats-plaster/deepsort/hiergat_data_v2512_2s/C0108_full_MP4_anno_for_labelling_fixed_full_temporal_2s/features',       # {1: 50, 3: 40, 4: 55, 6: 31}
#     '/root/vs-gats-plaster/deepsort/hiergat_data_v2512_2s/C0110_full_MP4_anno_for_labelling_full_temporal_2s/features',             # {1: 4, 2: 47, 3: 56, 4: 43}
    
#     # Previously used as validation
#     '/root/vs-gats-plaster/deepsort/hiergat_data_v2512_2s/C0100_full_MP4_anno_for_labelling_done_putu_full_temporal_2s/features', # {0: 41, 2: 18, 3: 1, 4: 3, 5: 1}
#     # '/root/vs-gats-plaster/deepsort/hiergat_data_v2512_2s/C0109_full_MP4_anno_for_labelling_full_temporal_2s/features', # OLD
#     '/root/vs-gats-plaster/deepsort/hiergat_data_v2601_2s/C0109_full_MP4_anno_for_labelling_full_temporal_2s/features',   # NEW: {1: 2512, 3: 78, 4: 68}
    
#     # Previously used as testing
#     '/root/vs-gats-plaster/deepsort/hiergat_data_v2512_2s/C0104_full_MP4_anno_for_labelling_full_temporal_2s/features',           # {}
#     '/root/vs-gats-plaster/deepsort/hiergat_data_v2512_2s/C0099_full_MP4_anno_for_labelling_done_arga_full_temporal_2s/features', # {0: 13, 2: 456, 5: 12, 6: 30}
# ]

# ================================
# Features for seq_len = 1 seconds
# ================================
# train_features_dirs = [
#     '/root/vs-gats-plaster/deepsort/hiergat_data_v2512_1s/C0071_full_MP4_anno_for_labelling_done_faridz_full_temporal_1s/features',
#     '/root/vs-gats-plaster/deepsort/hiergat_data_v2512_1s/C0074_full_MP4_anno_for_labelling_done_ray_full_temporal_1s/features',   
#     '/root/vs-gats-plaster/deepsort/hiergat_data_v2512_1s/C0078_full_MP4_anno_for_labelling_done_anne_full_temporal_1s/features',  
#     '/root/vs-gats-plaster/deepsort/hiergat_data_v2512_1s/C0083_full_MP4_anno_for_labelling_done_putu_full_temporal_1s/features',  
#     '/root/vs-gats-plaster/deepsort/hiergat_data_v2512_1s/C0085_full_MP4_anno_for_labelling_done_yoga_full_temporal_1s/features',  
#     '/root/vs-gats-plaster/deepsort/hiergat_data_v2512_1s/C0087_full_MP4_anno_for_labelling_done_arga_full_temporal_1s/features',  
#     '/root/vs-gats-plaster/deepsort/hiergat_data_v2512_1s/C0090_full_MP4_anno_for_labelling_rizky_full_temporal_1s/features',
#     '/root/vs-gats-plaster/deepsort/hiergat_data_v2512_1s/C0098_full_MP4_anno_for_labelling_done_akbar_full_temporal_1s/features', 
#     '/root/vs-gats-plaster/deepsort/hiergat_data_v2512_1s/C0101_full_MP4_anno_for_labelling_done_faridz_full_temporal_1s/features',
#     '/root/vs-gats-plaster/deepsort/hiergat_data_v2512_1s/C0105_full_MP4_anno_for_labelling_full_temporal_1s/features',
#     '/root/vs-gats-plaster/deepsort/hiergat_data_v2512_1s/C0106_full_MP4_anno_for_labelling_full_temporal_1s/features',
#     '/root/vs-gats-plaster/deepsort/hiergat_data_v2512_1s/C0108_full_MP4_anno_for_labelling_fixed_full_temporal_1s/features',      
#     '/root/vs-gats-plaster/deepsort/hiergat_data_v2512_1s/C0110_full_MP4_anno_for_labelling_full_temporal_1s/features',
    
#     # Previously used as validation
#     '/root/vs-gats-plaster/deepsort/hiergat_data_v2512_1s/C0100_full_MP4_anno_for_labelling_done_putu_full_temporal_1s/features',  
#     '/root/vs-gats-plaster/deepsort/hiergat_data_v2512_1s/C0109_full_MP4_anno_for_labelling_full_temporal_1s/features',            
    
#     # Previously used as testing
#     '/root/vs-gats-plaster/deepsort/hiergat_data_v2512_1s/C0104_full_MP4_anno_for_labelling_full_temporal_1s/features',
#     '/root/vs-gats-plaster/deepsort/hiergat_data_v2512_1s/C0099_full_MP4_anno_for_labelling_done_arga_full_temporal_1s/features',  
# ]

train_features_dirs = [Path(p) for p in train_features_dirs]

# Data

In [ ]:
def get_features_dirs_df(features_dirs):
    feature_dir_dicts = []
    for dirs in features_dirs:
        for dir in dirs.iterdir():
            action_label_str = str(dir).rsplit('_action_')[-1]
            is_interpolated = 0
            if '_interp' in action_label_str:
                action_label_str = action_label_str.replace('_interp', '')
                is_interpolated = 1
            action_label = int(action_label_str)
            action_class = action_classes[action_label]
            if 'new_action_classes' in globals():
                try:
                    action_label = new_action_classes.index(action_class) # type: ignore
                except ValueError:
                    action_label = -1
            feature_dir_dicts.append({
                'base_dir': str(dirs),
                'dir': str(dir),
                'action_label': action_label,
                # 'new_action_label': new_action_label,
                'is_interpolated': is_interpolated,
            })
    feature_dirs_df = pd.DataFrame(feature_dir_dicts)
    feature_dirs_df = feature_dirs_df[feature_dirs_df['action_label'] != -1] # remove rows where action_label == -1
    return feature_dirs_df

train_feature_dirs_df = get_features_dirs_df(train_features_dirs)

print("Training set action label counts:")
print(train_feature_dirs_df['action_label'].value_counts().sort_index().to_dict())

In [ ]:
import re

def create_data_df(feature_dirs, action_classes, new_action_classes=None, 
                   downsampling: int = 1, test_data: bool = False):
    base_dirs = []
    dirs = []
    start_fids = []
    end_fids = []
    human_features_list = []
    human_boxes_list = []
    human_poses_list = []
    object_features_list = []
    object_boxes_list = []
    gt_list = [] if not test_data else None
    action_labels = [] if not test_data else None
    xs_steps = []

    for feature_dir in feature_dirs:
        feature_dir = Path(feature_dir)
        
        # Store directories
        base_dirs.append(str(feature_dir.parent.parent))
        dirs.append(str(feature_dir))
        
        # Store start and end FIDs
        match = re.search(r"range_([0-9]+(?:\.[0-9]+)?)_([0-9]+(?:\.[0-9]+)?)", str(feature_dir))
        start_fid, end_fid = map(float, match.groups())
        start_fids.append(start_fid)
        end_fids.append(end_fid)
        
        # Load and store human (subject) features
        subject_visual_features = np.load(feature_dir / 'subject_visual_features.npy')
        subject_boxes = np.load(feature_dir / 'subject_boxes.npy')
        # subject_poses = np.zeros((subject_visual_features.shape[0], 17, 2))
        subject_poses = np.load(feature_dir / 'subject_poses.npy')
        
        human_features_list.append([subject_visual_features])
        human_boxes_list.append([subject_boxes])
        human_poses_list.append([subject_poses])
        
        # Load and store object features
        object_visual_features = np.load(feature_dir / 'object_visual_features.npy')
        object_boxes = np.load(feature_dir / 'object_boxes.npy')
        
        object_features_list.append(object_visual_features[:, np.newaxis, :])
        object_boxes_list.append(object_boxes[:, np.newaxis, :])
        
        # Extract and store ground-truth action label
        if not test_data:
            action_label_str = str(feature_dir).split('_action_')[-1]
            if '_' in action_label_str:
                action_label_str = action_label_str.split('_')[0]
            action_label = int(action_label_str)
            if isinstance(new_action_classes, (list, tuple)):
                action_class = action_classes[action_label]
                action_label = new_action_classes.index(action_class)
            seq_len = subject_visual_features.shape[0]
            gt_list.append({
                'Human1': [action_label] * seq_len,
            })
            action_labels.append(action_label)
        
        # Store number of steps
        num_steps = len(subject_visual_features[downsampling - 1::downsampling])
        xs_steps.append(num_steps)

    xs_steps = np.array(xs_steps, dtype=np.float32)

    data_df = pd.DataFrame({
        'human_features': human_features_list,
        'human_boxes': human_boxes_list,
        'human_poses': human_poses_list,
        'object_features': object_features_list,
        'object_boxes': object_boxes_list,
        'gt': gt_list,
        'xs_step': xs_steps,
        'base_dir': base_dirs,
        'dir': dirs,
        'start_fid': start_fids,
        'end_fid': end_fids,
        'action_label': action_labels,
    })
    data_df['xs_step'] = data_df['xs_step'].astype(int)
    data_df['start_fid'] = data_df['start_fid'].astype(int)
    data_df['end_fid'] = data_df['end_fid'].astype(int)
    return data_df

df = create_data_df(train_feature_dirs_df['dir'].tolist(), action_classes, new_action_classes if 'new_action_classes' in globals() else None)
df.head()

In [ ]:
for base_dir, group_df in df.groupby('base_dir'):
    print("================================================================================================================================")
    print(base_dir)
    print("================================================================================================================================")
    group_df['split'] = 'train'
    group_df = group_df.sort_values('start_fid')
    
    min_fid = 0
    max_fid = group_df['end_fid'].max()
    duration = max_fid - min_fid
    val_duration = int(duration * val_fraction)
    
    # Determine validation and testing FID ranges
    assert duration//2 >= val_duration+val_duration, f"Duration ({duration}) should be at least twice the validation duration ({val_duration})!"
    # val_start_fid = random.randrange(0, duration//2) # Sample a random FID from the start to the middle
    val_start_fid   = random.randrange(duration//2, duration-val_duration-val_duration) # Sample a random FID from the middle to the end
    val_end_fid     = val_start_fid + val_duration - 1
    test_start_fid  = val_start_fid + val_duration
    test_end_fid    = test_start_fid + val_duration - 1
    
    print(f"Validation FID range: {val_start_fid}-{val_end_fid}, length: {val_end_fid-val_start_fid+1}")
    print(f"Testing FID range: {test_start_fid}-{test_end_fid}, length: {test_end_fid-test_start_fid+1}")
    print()
    
    # Swap validation and testing FID ranges if validation is larger than testing
    base_dir_cond = df['base_dir'] == base_dir
    val_len = len(df.loc[base_dir_cond & (df['start_fid'] >= val_start_fid) & (df['end_fid'] <= val_end_fid)])
    test_len = len(df.loc[base_dir_cond & (df['start_fid'] >= test_start_fid) & (df['end_fid'] <= test_end_fid)])
    if val_len > test_len:
        tmp_start_fid  = test_start_fid
        tmp_end_fid    = test_end_fid
        test_start_fid = val_start_fid
        test_end_fid   = val_end_fid
        val_start_fid  = tmp_start_fid
        val_end_fid    = tmp_end_fid
        
    # Assign splits
    df.loc[base_dir_cond & (df['start_fid'] >= val_start_fid) & (df['end_fid'] <= val_end_fid), 'split'] = 'val'
    df.loc[base_dir_cond & (df['start_fid'] >= test_start_fid) & (df['end_fid'] <= test_end_fid), 'split'] = 'test'
    df.loc[base_dir_cond & (df['split'].isna()), 'split'] = 'train'
    
    train_group_df = df[base_dir_cond & (df['split'] == 'train')]
    val_group_df = df[base_dir_cond & (df['split'] == 'val')]
    test_group_df = df[base_dir_cond & (df['split'] == 'test')]
    
    try:
        assert len(test_group_df) > len(val_group_df) and len(df[base_dir_cond]) == len(train_group_df)+len(val_group_df)+len(test_group_df)
    except:
        print("len(df[base_dir_cond]):", len(df[base_dir_cond]))
        print("len(train_group_df):", len(train_group_df))
        print("len(val_group_df):", len(val_group_df))
        print("len(test_group_df):", len(test_group_df))
    print("Training action label info:")
    print(train_group_df['action_label'].value_counts())
    print("Total:", len(train_group_df))
    print()
    print("Validation action label info:")
    print(val_group_df['action_label'].value_counts())
    print("Total:", len(val_group_df))
    print()
    print("Testing action label info:")
    print(test_group_df['action_label'].value_counts())
    print("Total:", len(test_group_df))
    print()

In [ ]:
train_df = df[df['split'] == 'train']
val_df = df[df['split'] == 'val']
test_df = df[df['split'] == 'test']

train_size = len(train_df)
val_size = len(val_df)
test_size = len(test_df)
all_size = len(df)

print(f"Total train data: {train_size} ({train_size/all_size*100:.2f}%)")
print(f"Total val data: {val_size} ({val_size/all_size*100:.2f}%)")
print(f"Total test data: {test_size} ({test_size/all_size*100:.2f}%)")

In [ ]:
data_cols  = ['human_features', 'human_boxes', 'human_poses', 'object_features', 'object_boxes', 'gt', 'xs_step']
train_data = train_df[data_cols].to_numpy().T.tolist()
val_data   = val_df[data_cols].to_numpy().T.tolist()
test_data  = test_df[data_cols].to_numpy().T.tolist()

In [ ]:
test_video_features_dir = '/root/vs-gats-plaster/deepsort/hiergat_data_v3_3s/C0074_full_MP4_anno_for_labelling_done_ray_full_temporal_3s'

test_video_w_seen_df = df[df['base_dir'] == test_video_features_dir]
test_video_unseen_df = test_df[test_df['base_dir'] == test_video_features_dir]

test_video_w_seen_data = test_video_w_seen_df[data_cols].to_numpy().T.tolist()
test_video_unseen_data = test_video_unseen_df[data_cols].to_numpy().T.tolist()

print("Test video action label counts (with seen data):")
print(test_video_w_seen_df['action_label'].value_counts().sort_index())
print()
print("Test video action label counts (only unseen data):")
print(test_video_unseen_df['action_label'].value_counts().sort_index())

In [ ]:
train_loader, scalers, _ = create_data_loader(
    *train_data, 
    model_name, 
    batch_size=batch_size, 
    shuffle=True,
    scaling_strategy=scaling_strategy, 
    sigma=sigma,
    downsampling=downsampling,
)
val_loader, _, _ = create_data_loader(
    *val_data, 
    model_name, 
    batch_size=len(val_data[0]),
    shuffle=False, 
    scalers=scalers, 
    sigma=sigma, 
    downsampling=downsampling,
)
test_loader, _, _ = create_data_loader(
    *test_data, 
    model_name, 
    batch_size=len(test_data[0]),
    shuffle=False, 
    scalers=scalers, 
    sigma=sigma, 
    downsampling=downsampling,
)
test_video_w_seen_loader, _, _ = create_data_loader(
    *test_video_w_seen_data, 
    model_name, 
    batch_size=len(test_video_w_seen_data[0]),
    shuffle=False, 
    scalers=scalers, 
    sigma=sigma, 
    downsampling=downsampling,
)
test_video_unseen_loader, _, _ = create_data_loader(
    *test_video_unseen_data, 
    model_name, 
    batch_size=len(test_video_unseen_data[0]),
    shuffle=False, 
    scalers=scalers, 
    sigma=sigma, 
    downsampling=downsampling,
)
input_size = input_size_from_data_loader(train_loader, model_name, model_input_type)
data_info = {'input_size': input_size}

# Model

In [ ]:
model_creation_args = cfg.models.parameters
model_creation_args = {**data_info, **model_creation_args.__dict__}
dataset_name = cfg.data.name
model_creation_args['num_classes'] = (num_classes, None)
device = 'cuda' if torch.cuda.is_available() and cfg.resources.use_gpu else 'cpu'
model = TGGCN(feat_dim=1024, **model_creation_args).to(device)
if misc_dict.get('pretrained', False) and misc_dict.get('pretrained_path') is not None:
    state_dict = load_model_weights(misc_dict['pretrained_path'])
    model.load_state_dict(state_dict, strict=False)
params = model.parameters()
optimizer = torch.optim.Adam(params, lr=cfg.models.optimization.learning_rate)
criterion, loss_names = select_loss(model_name, model_input_type, dataset_name, cfg=cfg)
mtll_model = None
if misc_dict.get('multi_task_loss_learner', False):
    loss_types = select_loss_types(model_name, dataset_name, cfg=cfg)
    mask = select_loss_learning_mask(model_name, dataset_name, cfg=cfg)
    mtll_model = MultiTaskLossLearner(loss_types=loss_types, mask=mask).to(device)
    optimizer.add_param_group({'params': mtll_model.parameters()})
# Some config + model training
tensorboard_log_dir = cfg.models.logging.root_log_dir
checkpoint_name = cfg.models.logging.checkpoint_name
fetch_model_data = select_model_data_fetcher(model_name, model_input_type,
                                             dataset_name=dataset_name, **{**misc_dict, **cfg.models.parameters.__dict__})
feed_model_data = select_model_data_feeder(model_name, model_input_type, dataset_name=dataset_name, **misc_dict)
num_main_losses = decide_num_main_losses(model_name, dataset_name, {**misc_dict, **cfg.models.parameters.__dict__})

# Training

In [ ]:
# checkpoint = train(
#     model, 
#     train_loader, 
#     optimizer, 
#     criterion, 
#     cfg.models.optimization.epochs, 
#     device, 
#     loss_names,
#     clip_gradient_at=cfg.models.optimization.clip_gradient_at,
#     fetch_model_data=fetch_model_data, feed_model_data=feed_model_data,
#     val_loader=val_loader, 
#     mtll_model=mtll_model, 
#     num_main_losses=num_main_losses,
#     tensorboard_log_dir=tensorboard_log_dir, 
#     checkpoint_name=checkpoint_name,
# )
# # Logging
# if cfg.models.logging.log_dir is not None:
#     log_dir = cfg.models.logging.log_dir
#     checkpoint['scalers'] = scalers
#     save_checkpoint(log_dir, checkpoint, checkpoint_name=checkpoint_name, include_timestamp=False)

# Predict

In [ ]:
hiergat_checkpoint_path = Path('/root/workspace/HierGAT-Custom/outputs_hiergat/custom/hs512_e10_bs32_lr0.0001_0.5_20260119133043/hs512_e10_bs32_lr0.0001_0.5_20260119133043.tar')

hiergat_checkpoint = torch.load(hiergat_checkpoint_path, map_location=device)
hiergat_scalers = hiergat_checkpoint.get('scalers', None)
model_args = {
    # Original
    'add_segment_length': 0,
    'add_time_position': 0,
    'attention_style': 'v3',
    'bias': True,
    'cat_level_states': 0,
    'discrete_networks_num_layers': 1,
    'discrete_optimization_strategy': 'gs',
    'filter_discrete_updates': False,
    'gcn_node': 25,
    'hidden_size': 512,
    'input_size': (1124, 1024),
    'message_aggregation': 'att',
    'message_geometry_to_human': True,
    'message_geometry_to_objects': True,
    'message_granularity': 'v1',
    'message_human_to_objects': True,
    'message_humans_to_human': False,
    'message_objects_to_human': True,
    'message_objects_to_object': False,
    'message_segment': True,
    'message_type': 'v2',
    'num_classes': (num_classes, None),
    'object_segment_update_strategy': 'ind',
    'positional_encoding_style': 'e',
    'share_level_mlps': 0,
    'time_position_strategy': 's',
    'update_segment_threshold': 0.5,
    
    # Custom
    'feat_dim': 1024,
}
model = TGGCN(**model_args).to(device)
model.load_state_dict(hiergat_checkpoint['model_state_dict'], strict=False)
model.eval();

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix

from predict import match_shape, match_att_shape

inspect_model = False

model.eval();

## All Videos (Only Unseen)

In [ ]:
outputs, targets, attentions = [], [], []
for i, dataset in enumerate(test_loader):
    data, target = fetch_model_data(dataset, device=device)
    with torch.no_grad():
        output = feed_model_data(model, data)
    if inspect_model:
        output, attention_scores = output
        attention_scores = [att_score[:, 0] for att_score in attention_scores]
    if num_main_losses is not None:
        output = output[-num_main_losses:]
        target = target[-num_main_losses:]
    if downsampling > 1:
        for i, (out, tgt) in enumerate(zip(output, target)):
            if out.ndim != 4:
                raise RuntimeError(f'Number of dimensions for output is {out.ndim}')
            out = torch.repeat_interleave(out, repeats=downsampling, dim=-2)
            out = match_shape(out, tgt)
            output[i] = out
        if inspect_model:
            a_target = target[0]
            attention_scores = [torch.repeat_interleave(att_score, repeats=downsampling, dim=-2)
                                for att_score in attention_scores]
            attention_scores = [match_att_shape(att_score, a_target) for att_score in attention_scores]
            attentions.append(attention_scores)
    outputs += output
    targets += target

y_pred = torch.argmax(outputs[0], dim=1).cpu().numpy()
y_true = targets[0].squeeze(-1).mode(dim=1).values.cpu().numpy()

acc = accuracy_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred, average='weighted', zero_division=0)
precision = precision_score(y_true, y_pred, average='weighted', zero_division=0)
recall = recall_score(y_true, y_pred, average='weighted', zero_division=0)

print("Accuracy:", acc)
print("F1 Score:", f1)
print("Precision:", precision)
print("Recall:", recall)

In [ ]:
vis_new_action_classes = [
    # rebar
    'tie',              # 0
    'erect',            # 1
    'prepare',          # 2
    'transport',        # 3

    # equipment         
    'use',              # 4
    'carry',            # 5

    # all
    'inspect',          # 6
    'no interaction',   # 7
]

# ticklabels = action_classes
ticklabels = new_action_classes
if 'new_action_classes' in globals() and isinstance(new_action_classes, (list, tuple)): # type: ignore
    y_true = [ticklabels.index(new_action_classes[y]) for y in y_true] # type: ignore
    y_pred = [ticklabels.index(new_action_classes[y]) for y in y_pred] # type: ignore
    
cm = confusion_matrix(y_true, y_pred, labels=list(range(len(ticklabels))))

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=vis_new_action_classes, yticklabels=vis_new_action_classes)
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix")
plt.show()

## Single Video (Seen + Unseen)

In [ ]:
print("Selected test video ID:", Path(test_video_features_dir).stem[:5])

In [ ]:
outputs, targets, attentions = [], [], []
for i, dataset in enumerate(test_video_w_seen_loader):
    data, target = fetch_model_data(dataset, device=device)
    with torch.no_grad():
        output = feed_model_data(model, data)
    if inspect_model:
        output, attention_scores = output
        attention_scores = [att_score[:, 0] for att_score in attention_scores]
    if num_main_losses is not None:
        output = output[-num_main_losses:]
        target = target[-num_main_losses:]
    if downsampling > 1:
        for i, (out, tgt) in enumerate(zip(output, target)):
            if out.ndim != 4:
                raise RuntimeError(f'Number of dimensions for output is {out.ndim}')
            out = torch.repeat_interleave(out, repeats=downsampling, dim=-2)
            out = match_shape(out, tgt)
            output[i] = out
        if inspect_model:
            a_target = target[0]
            attention_scores = [torch.repeat_interleave(att_score, repeats=downsampling, dim=-2)
                                for att_score in attention_scores]
            attention_scores = [match_att_shape(att_score, a_target) for att_score in attention_scores]
            attentions.append(attention_scores)
    outputs += output
    targets += target
    
y_pred = torch.argmax(outputs[0], dim=1).cpu().numpy()
y_true = targets[0].squeeze(-1).mode(dim=1).values.cpu().numpy()

acc = accuracy_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred, average='weighted', zero_division=0)
precision = precision_score(y_true, y_pred, average='weighted', zero_division=0)
recall = recall_score(y_true, y_pred, average='weighted', zero_division=0)

print("Accuracy:", acc)
print("F1 Score:", f1)
print("Precision:", precision)
print("Recall:", recall)

In [ ]:
vis_new_action_classes = [
    # rebar
    'tie',              # 0
    'erect',            # 1
    'prepare',          # 2
    'transport',        # 3

    # equipment         
    'use',              # 4
    'carry',            # 5

    # all
    'inspect',          # 6
    'no interaction',   # 7
]

# ticklabels = action_classes
ticklabels = new_action_classes
if 'new_action_classes' in globals() and isinstance(new_action_classes, (list, tuple)): # type: ignore
    y_true = [ticklabels.index(new_action_classes[y]) for y in y_true] # type: ignore
    y_pred = [ticklabels.index(new_action_classes[y]) for y in y_pred] # type: ignore
    
cm = confusion_matrix(y_true, y_pred, labels=list(range(len(ticklabels))))

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=vis_new_action_classes, yticklabels=vis_new_action_classes)
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix")
plt.show()

## Single Video (Only Unseen)

In [ ]:
print("Selected test video ID:", Path(test_video_features_dir).stem[:5])

In [ ]:
outputs, targets, attentions = [], [], []
for i, dataset in enumerate(test_video_unseen_loader):
    data, target = fetch_model_data(dataset, device=device)
    with torch.no_grad():
        output = feed_model_data(model, data)
    if inspect_model:
        output, attention_scores = output
        attention_scores = [att_score[:, 0] for att_score in attention_scores]
    if num_main_losses is not None:
        output = output[-num_main_losses:]
        target = target[-num_main_losses:]
    if downsampling > 1:
        for i, (out, tgt) in enumerate(zip(output, target)):
            if out.ndim != 4:
                raise RuntimeError(f'Number of dimensions for output is {out.ndim}')
            out = torch.repeat_interleave(out, repeats=downsampling, dim=-2)
            out = match_shape(out, tgt)
            output[i] = out
        if inspect_model:
            a_target = target[0]
            attention_scores = [torch.repeat_interleave(att_score, repeats=downsampling, dim=-2)
                                for att_score in attention_scores]
            attention_scores = [match_att_shape(att_score, a_target) for att_score in attention_scores]
            attentions.append(attention_scores)
    outputs += output
    targets += target
    
y_pred = torch.argmax(outputs[0], dim=1).cpu().numpy()
y_true = targets[0].squeeze(-1).mode(dim=1).values.cpu().numpy()

acc = accuracy_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred, average='weighted', zero_division=0)
precision = precision_score(y_true, y_pred, average='weighted', zero_division=0)
recall = recall_score(y_true, y_pred, average='weighted', zero_division=0)

print("Accuracy:", acc)
print("F1 Score:", f1)
print("Precision:", precision)
print("Recall:", recall)

In [ ]:
vis_new_action_classes = [
    # rebar
    'tie',              # 0
    'erect',            # 1
    'prepare',          # 2
    'transport',        # 3

    # equipment         
    'use',              # 4
    'carry',            # 5

    # all
    'inspect',          # 6
    'no interaction',   # 7
]

# ticklabels = action_classes
ticklabels = new_action_classes
if 'new_action_classes' in globals() and isinstance(new_action_classes, (list, tuple)): # type: ignore
    y_true = [ticklabels.index(new_action_classes[y]) for y in y_true] # type: ignore
    y_pred = [ticklabels.index(new_action_classes[y]) for y in y_pred] # type: ignore
    
cm = confusion_matrix(y_true, y_pred, labels=list(range(len(ticklabels))))

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=vis_new_action_classes, yticklabels=vis_new_action_classes)
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix")
plt.show()